# Introduction to the JWST pipeline

This is a tutorial on running the JWST Science Calibration Pipeline (referred to as “the pipeline”).


## TOC:
* [Overview of JWST pipeline](#1)
    * [What is the JWST Pipeline?](#1a)
    * [Stages of the Pipeline](#1b)
* [Reference Files and CRDS](#3)
* [Parameter Files](#4)
* [Installation](#2)
    * [Creating an Envrionment](#2a)
    * [Installing the Last Released Verion of JWST](#2b)
    * [Installing the Latest Development Version from Github](#2c)
* [Running the Pipeline in Python](#5)
    * [Example 1: Running Detector1Pipeline on a NIRCAM Image](#5a)
    * [Example 2: Running a Single Pipeline Step (LinearityStep) on a NIRCAM Image](#5b)
    * [Options when Running the Pipeline in Python: 'call' vs 'run' methods](#5c)
    * [Example 3: Using 'run' Instead of 'call' to run Detector1Pipeline on NIRCAM image](#5d)
* [Configuring the Pipeline in Python](#6)
    * [Example 4: Setting Parameters Using a Custom Parameter File](#6)
    * [Example 5: Setting Parameters with a Dictionary when Constructing a Pipeline/Step Object](#6)
    * [Example 6: Changing Reference Files](#6)
    * [Example 7: Skipping Pipeline Steps](#6)
    * [Example 8: Confguring the Pipeline Object Directly (not reccomended)](#6)
* [Running the Pipeline using the Command Line Interface](#7)
    * [Example 9: Running Detector1Pipeline on a NIRCAM Image](#7a)
    * [Example 10: Running a Single Pipeline Step (LinearityStep) on a NIRCAM Image](#7b)

## Overview of JWST pipeline <a class="anchor" id="1"></a>

The following will provide a very brief summary to provide context for some of the information in this notebook: a more extensive description is available in the [JWST pipeline documentation](https://jwst-pipeline.readthedocs.io/en/latest/).

#### What is the JWST pipeline?<a class="anchor" id="1a"></a>

The JWST pipeline is the software suite that processes raw data from the JWST instruments to make it science-ready by performing various corrections and operations on the data. The pipeline is written in Python and hosted on [Github]((https://github.com/spacetelescope/jwst).

Data products available for download in the MAST archive have already been processed with this pipeline, but users wishing to do custom processing may want to configure and run them pipeline themselves. The pipeline can be ran and configured in [Python](#5) or via a command line interface using the `strun` command. 

The JWST pipeline is subdivided into three stages (e.g `Detector1Pipeline`, `Image3Pipeline`). Each dataset, depending on what type of observation it is, will have three stages in its pipeline. Furthermore, each pipeline consists of individual Steps (e.g AssignWCSStep, LinearityStep). The modular nature of the pipeline allows users to customize their processing by running the full pipeline, a single stage, a single step within each stage, etc. Steps can also be 'switched' on and off during processing (for example, running the `Detector1Pipeline` but omitting the `DarkCurrentStep`.

The main way in which the pipeline is configured is via `Parameter Files`. `Reference Files` provide the data for the multitude of calibrations and corrections performed by the pipeline. 

#### Stages of the Pipeline<a class="anchor" id="1b"></a>
The JWST pipeline is broken into 3 [stages](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html). Each stage consists of a series of steps. 

- Stage 1: Detector-level corrections and ramp fitting for individual exposures

  The first stage, called `calwebb_detector1`, is applied nearly universally for all instruments and modes. It consists of detector-level corrections that are performed on a group-by-group basis, followed by ramp fitting. The output of stage 1 processing is a countrate image per exposure, or per integration for some modes. The input to this step is `uncal.fits` file, and the output is a `rate.fits` file.


- Stage 2: Instrument-mode calibrations for individual exposures

  The second stage is split into separate modules for imaging and spectroscopic data, called `calwebb_image2` and `calwebb_spec2` respectively. It consists of additional instrument-level and observing-mode corrections and calibrations to produce fully calibrated exposures. The details differ for imaging and spectroscopic exposures, and there are some corrections that are unique to certain instruments or modes. The input to this step is `rate.fits` file, and the output is a `cal.fits` file.
  
  
- Stage 3: Combining data from multiple exposures within an observation

  The third stage is divided into five separate modules depending on the observing mode. It consists of routines that work with multiple exposures and in most cases produce some kind of combined product. There are unique pipeline modules for stage 3 processing of imaging, spectroscopic, coronagraphic, AMI, and TSO observations.


# Installation <a class="anchor" id="2"></a>

#### Creating an Envrionment <a class="anchor" id="2a"></a>

For the purpose of this tutorial we will install jwst in a new conda environment, called `jwst-test` using Python 3.8. This tutorial assumes a user has conda installed.
The package installs its dependencies but we need to install Python and numpy in the conda environment.  

- Make and activate the new conda environment

*% conda create -n jwst-test python=3.8*

*% source activate jwst-test*

- Install numpy

*% pip install numpy==1.19*

- Another useful (not required) package is jupyter. It installs ipython and jupyter notebook.

*% pip install jupyter*


#### Installing the Last Released Verion of JWST <a class="anchor" id="2b"></a>

The latest public release can be installed with one command.

*% pip install jwst*

#### Installing the Latest Development Version from Github <a class="anchor" id="2c"></a>

To work with the absolute most up-to-date version of the pipeline, the `main` branch of the Github repository can be installed. The source code for the JWST pipeline is located on Github in the spacetelescope organization.

https://github.com/spacetelescope/jwst

To install the latest code from github use the command:

*% pip install git+https://github.com/spacetelescope/jwst*

## Reference Files and CRDS <a class="anchor" id="3"></a>

Reference files are data files - seperate from the input data being processed - that the JWST pipeline requires to calibrate data and apply necessary corrections (e.g dark correction, bias correction) and calibrations to achieve science-ready data. Each data set has a specific set of up-to-date reference files associated with it which supply the data for all the pipeline calibration steps.

Reference files are created and validated by the JWST instrument teams. Because many of these corrections are time dependent (e.g a monthly dark file), or are periodically updated and improved as understanding of the instrument improves, they must be version controlled to ensure users can access the exact set of files for a dataset as well as revert back to previous versions if needed. Managing these files and determining the exact set of reference files to apply to any given data file is not a trivial task: the CRDS (Calibration Reference Data System) manages these intricacies and is the interface for obtaining and managing pipeline reference files.

Users internal to the STScI network already have all reference files on disk in the CRDS file system on central storage and will not have to do any special setup to allow the pipeline access to reference files. External users will have to configure their environment to point to the CRDS_SERVER_URL (to download reference files from this server) as well as a directory in their local filesystem (CRDS_PATH) to which these files will be downloaded to, and which the pipeline will recognize as the location of these files when run. In your shell, set the environment variables:

##### *% export CRDS_PATH=$HOME/crds_cache*

##### *% export CRDS_SERVER_URL=https://jwst-crds.stsci.edu*

When you run the pipeline on a dataset for the first time, the CRDS code within the pipeline will determine the best set of reference files based on the exposure date, observation parameters (e.g instrument type and mode, filter), and 'context' (akin to a snapshot in time, the most up-to-date being the default, but users can also roll back to a previous context with different mappings). Then, it will connect to the server (specified via `CRDS_SERVER_URL`) and download those files to your local directory `CRDS_PATH`. Subsequent runs of the Python will first check `CRDS_PATH` for existing required reference files so they aren't re-downloaded. 



# Parameter Files <a class="anchor" id="4"></a>

Parameter files are [ASDF](https://readthedocs.org/projects/asdf/) format files that set the multitude of parameters for pipeline steps, and are the main way that the JWST pipeline is configured. They are similar to reference files in the sense that they are created by JWST instrument teams and direct the pipeline behavior/output. Also like reference files, they are version controlled and managed by CRDS. This version-controlling gives the instrument teams the ability to evolve parameter values for different observing modes over time.

An example of a parameter file is:

        >> jwst_miri_pars-detector1pipeline_0001.asdf

When a user runs the pipeline, the corresponding parameter files for the step(s) being run and the dataset being processed will be downloaded to the local crds cache (CRDS_PATH) along side the other reference files. 

Note that not every observing mode/pipeline step have a parameter file - in some cases, there are no special parameter selections needed and the coded defaults suffice. For example, the `Image2Pipeline` for Nircam does not have a parameter file, and the coded default values are used for every dataset. See the [JWST CRDS website](https://jwst-crds.stsci.edu/) to browse parameter files for different steps and observing modes.

Parameter files may be ignored or overridden when running the pipeline. Users can override parameters individually, or supply their own custom parameter file which will override keywords in any parameter file existing in CRDS for that dataset. See \<link> for more information on configuring the pipeline. 


# Running the Pipeline in Python <a class="anchor" id="5"></a>

The following section will demonstrate several possibilities of running and configuring the pipeline/pipeline step using the Python interface.

#### Example 1: Running Detector1Pipeline on a NIRCAM image.<a class="anchor" id="5a"></a>

The Detector1Pipeline performs many of the basic detector-level corrections as well as ramp fitting. Start by importing ``Detector1Pipeline`` from ``jwst.pipeline``

In [54]:
from jwst.pipeline import Detector1Pipeline

Now create an instance of the ``Detector1Pipeline`` - we'll name this object `pipe` here.

In [6]:
pipe = Detector1Pipeline()

2021-03-31 22:29:48,223 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-03-31 22:29:48,225 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-03-31 22:29:48,227 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-03-31 22:29:48,229 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-03-31 22:29:48,231 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-03-31 22:29:48,233 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-03-31 22:29:48,235 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-03-31 22:29:48,237 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-03-31 22:29:48,239 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2021-03-31 22:29:48,241 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2021-03-31 2

And now, in the most simple use case, process the `uncal` fits file in this directory using the `pipe.call` method. This will return an `ImageModel` representing the calibrated data. Additionally, you'll see the file `jw42424001001_01101_00001_nrca5_trapsfilled.fits` output in the cwd - this is an intermediate output from the [persistance](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/nonscience_products.html#charge-trap-state-data-trapsfilled) step of the pipeline. We will save the returned image model to the variable `result`. By setting `save_results` to True, we will also get this same data model written out to a fits file in the cwd (jw42424001001_01101_00001_nrca5_rate.fits, a `rate` file is the output product from the Detector1Pipeline). There is also an optional rguement `output_file` where you can choose the name of your output file, but in this case we will just used the default naming convention which takes the name of the input `uncal` file and changes the suffix to `rate`.

In [100]:
result = pipe.call('jw42424001001_01101_00001_nrca5_uncal.fits', save_results=True)

2021-04-05 00:45:04,293 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-04-05 00:45:04,714 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_pars-detector1pipeline_0002.asdf
2021-04-05 00:45:04,723 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/asdf/asdf.py:212: AsdfWarning: File 'file:///Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_pars-detector1pipeline_0002.asdf' was created with extension 'asdf.extension.BuiltinExtension' from package asdf-2.7.2, but older version asdf-2.7.1 is installed
  warn

2021-04-05 00:45:05,235 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw42424001001_01101_00001_nrca5_uncal.fits' reftypes = ['dark', 'gain', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2021-04-05 00:45:05,242 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_dark_0043.fits'.
2021-04-05 00:45:05,243 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_gain_0056.fits'.
2021-04-05 00:45:05,244 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_linearity_0052.fits'.
2021-04-05 00:45:05,245 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_

2021-04-05 00:45:07,654 - stpipe.Detector1Pipeline.refpix - INFO - use_side_ref_pixels = True
2021-04-05 00:45:07,655 - stpipe.Detector1Pipeline.refpix - INFO - odd_even_columns = True
2021-04-05 00:45:07,655 - stpipe.Detector1Pipeline.refpix - INFO - side_smoothing_length = 11
2021-04-05 00:45:07,656 - stpipe.Detector1Pipeline.refpix - INFO - side_gain = 1.000000
2021-04-05 00:45:07,657 - stpipe.Detector1Pipeline.refpix - INFO - odd_even_rows = True
2021-04-05 00:45:09,500 - stpipe.Detector1Pipeline.refpix - INFO - Step refpix done
2021-04-05 00:45:09,631 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(1, 7, 2048, 2048) from jw42424001001_01101_00001_nrca5_uncal.fits>,).
2021-04-05 00:45:09,633 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False,

2021-04-05 00:45:51,327 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<ImageModel(2048, 2048) from jw42424001001_01101_00001_nrca5_uncal.fits>,).
2021-04-05 00:45:51,329 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'gain_scale', 'search_output_file': True, 'input_dir': ''}
2021-04-05 00:45:51,384 - stpipe.Detector1Pipeline.gain_scale - INFO - GAINFACT not found in gain reference file
2021-04-05 00:45:51,385 - stpipe.Detector1Pipeline.gain_scale - INFO - Step will be skipped
2021-04-05 00:45:51,386 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale done
2021-04-05 00:45:51,539 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<CubeModel(1, 2048, 2048) from jw42424001001_01101_0

The calibrated data model contains information about how the pipeline was run - for example, you can see in `model.meta.cal_step` which steps were run (COMPLETE) and which were not (SKIPPED).

In [56]:
result.meta.cal_step.instance

2021-04-01 02:05:01,448 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



{'dark_sub': 'COMPLETE',
 'dq_init': 'COMPLETE',
 'group_scale': 'SKIPPED',
 'jump': 'COMPLETE',
 'linearity': 'COMPLETE',
 'persistence': 'COMPLETE',
 'refpix': 'COMPLETE',
 'saturation': 'COMPLETE',
 'superbias': 'COMPLETE',
 'ramp_fit': 'COMPLETE',
 'gain_scale': 'SKIPPED'}

You can also inspect the names of the reference files that were used:

In [50]:
result.meta.ref_file.instance

{'crds': {'sw_version': '10.3.5', 'context_used': 'jwst_0704.pmap'},
 'dark': {'name': 'crds://jwst_nircam_dark_0043.fits'},
 'gain': {'name': 'crds://jwst_nircam_gain_0056.fits'},
 'linearity': {'name': 'crds://jwst_nircam_linearity_0052.fits'},
 'mask': {'name': 'crds://jwst_nircam_mask_0038.fits'},
 'persat': {'name': 'crds://jwst_nircam_persat_0016.fits'},
 'readnoise': {'name': 'crds://jwst_nircam_readnoise_0042.fits'},
 'saturation': {'name': 'crds://jwst_nircam_saturation_0063.fits'},
 'superbias': {'name': 'crds://jwst_nircam_superbias_0022.fits'},
 'trapdensity': {'name': 'crds://jwst_nircam_trapdensity_0004.fits'},
 'trappars': {'name': 'crds://jwst_nircam_trappars_0002.fits'}}

#### Example 2: Running a Single Pipeline Step (LinearityStep) on a NIRCAM image <a class="anchor" id="5b"></a>

In the first example, the full Detector1Pipeline was run on the input `uncal` date. We can run the steps that comprise this pipeline individually, for example, the `Linearity Step`. Running a pipeline step is done in the exact same way: import the step, create an instance of it, and use the `call` method. In this case, we will not set `save_results` to True so no intermediate output will be written, only a `DataModel` with `LinearityStep` will be returned in memory.

In [76]:
from jwst.linearity import LinearityStep

step = LinearityStep()

lin_corr_im = step.call('jw42424001001_01101_00001_nrca5_uncal.fits')

2021-04-01 23:29:52,221 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-04-01 23:29:52,227 - stpipe.LinearityStep - INFO - LinearityStep instance created.
2021-04-01 23:29:52,525 - stpipe.LinearityStep - INFO - LinearityStep instance created.
2021-04-01 23:29:52,751 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/stpipe/step.py:359: ResourceWarning: unclosed file <_io.FileIO name='jw42424001001_01101_00001_nrca5_uncal.fits' mode='rb' closefd=True>
  gc.collect()

2021-04-01 23:29:52,800 - stpipe.LinearityStep - INFO - Step LinearityStep running with args ('jw42424001001_01101_00001_nrca

#### Options when Running the Pipeline in Python: 'call' vs 'run' methods <a class="anchor" id="5c"></a>

In the above examples, we used the `pipe.call` method to run the pipeline. If you look at the available methods, you may be confused to see `pipe.call` and `pipe.run` - both of these names imply that when called, they should process the data. There is, however, a distinction. For example, if you create a `Detector1Pipeline` instance like the above example,

    >> pipe = Detector1Pipeline()
    
`pipe.run('input_file.fits)` or, equivilantly 
    

    

Both `call` and `run` (as well as the private `__call__`) eventually execute the `run` method, but `call` does additional setup. The main distinction is that `run` uses the class you instantiated, so you can directly modify parameters that are represented as attributes of this class, while `call` creates a new class of the same type internally and relies on parameter files to direct processing. Because `run` is on the class itself, `pipe.run` is equivilant to calling the class directly `pipe()`. The `call` method will look in CRDS for a parameter file, or you may pass a local parameter file that you've customized as input. 

`call` is configurable via parameter files, `run` is configurable via setting the attributes directly. The only time `run` is suggested is if you don't wish to use parameter files at all. It is generally recommended that `call`, and therefore CRDS or custom parameter files, is used when running the pipeline.

#### Example 3: Using `run` Instead of `call` to run Detector1Pipeline on NIRCAM image.  <a class="anchor" id="5d"></a>
In this example, we will use the `run` method to run the `Image2Pipeline`. Parameters are set directly on the insance of `Image2Pipeline`, config files aren't used.

In [82]:
from jwst.pipeline import Image2Pipeline

# save_results has to be set here, because now we're calling this class directly 
pipe2 = Image2Pipeline(save_results=True)

2021-04-02 00:02:21,840 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-04-02 00:02:21,847 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2021-04-02 00:02:21,849 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-04-02 00:02:21,851 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-04-02 00:02:21,853 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2021-04-02 00:02:21,855 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2021-04-02 00:02:21,857 - stpipe.Image2Pipeline.resample - INFO - ResampleStep inst

In [117]:
result2 = pipe2.run('jw42424001001_01101_00001_nrca5_rate.fits')

2021-04-05 09:58:59,676 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('jw42424001001_01101_00001_nrca5_rate.fits',).


ValidationError: Config parameter 'suffix': the value "False" is of the wrong type.

Or, identically, calling the pipeline obj directly which invokes the `run` method.

In [116]:
result2 = pipe2('jw42424001001_01101_00001_nrca5_rate.fits')

2021-04-05 09:58:51,342 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('jw42424001001_01101_00001_nrca5_rate.fits',).


ValidationError: Config parameter 'suffix': the value "False" is of the wrong type.

## Configuring the Pipeline in Python<a class="anchor" id="6"></a>

As mentioned above, the parameter file is the basis of how pipelines/steps are configured - the parameter files in CRDS correspond to the 'best' set of parameters as chosen by the JWST instrument teams - but users are free to change any of these values when running the pipeline. There are a number of ways to do this when running the pipeline in Python.

Because there are multiple locations in which a parameter value may be specified (parameter file, coded defaults, setting the attributes on the pipe/step class directly, and command line paramater overriding which will be introduced later), the pipeline uses hierarchy rules to determine which value are used if they are specified in multiple locations.

1. Value specified in command line, or set on class directly when using ``pipe.run()`` or `pipe()`.
2. Value in user-provided parameter file.
3. Value in CRDS parameter file.
4. `Step`-coded default, determined by the parameter definition `Step.spec`

Let's go through a few examples of this. The output `rate` file from the last example can be run through the [Image2Pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html), which applies several calibtration steps including flat fielding and assigning a WCS. When this pipeline is run, the resulting data product is a `cal` file.

Again, begin by importing the desired pipeline or step, in this case `Image3Pipeline`, and creating an instance of it `pipe3`:

In [101]:
from jwst.pipeline import Image3Pipeline
pipe3 = Image3Pipeline()

2021-04-05 00:48:02,899 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



If we run it just like this without making any modifications, the corresponding parameter file will be obtained from CRDS and those values will be used. If you look at the logging output, you'll see one of the first lines indicates that the file `jwst_nircam_pars-tweakregstep_0006.asdf` is being used.

In [127]:
result3 = pipe3.call('jw42424001001_01101_00001_nrca5_cal.fits')

2021-04-09 00:21:08,054 - stpipe.Image2Pipeline - INFO - PARS-TWEAKREGSTEP parameters found: /Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_pars-tweakregstep_0006.asdf
2021-04-09 00:21:08,068 - stpipe.Image2Pipeline - INFO - PARS-TWEAKREGSTEP parameters are {'brightest': 100, 'catalog_format': 'ecsv', 'enforce_user_order': False, 'expand_refcat': False, 'fitgeometry': 'general', 'kernel_fwhm': 2.302, 'minobj': 15, 'nclip': 3, 'save_catalogs': False, 'searchrad': 1.0, 'separation': 0.5, 'sigma': 3.0, 'snr_threshold': 10, 'tolerance': 1.0, 'use2dhist': True, 'xoffset': 0.0, 'yoffset': 0.0}
2021-04-09 00:21:08,137 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2021-04-09 00:21:08,138 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2021-04-09 00:21:08,141 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2021-04-09 00:21:08,145 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2

TypeError: Input <ImageModel(2048, 2048) from jw42424001001_01101_00001_nrca5_cal.fits> is not a list of DataModels or an ASN file

In the cwd, we have the file `'my_custom_parfile_tweakregstep.asdf'`. This file is in the exact same format as `jwst_nircam_pars-tweakregstep_0006.asdf`, but some of the values for the tweakreg step have been changed. We can direct `Image3Pipeline` to run with these values instead by setting the `config_file` argument to the path of our custom refence file, in this case just config_file = my_custom_parfile_tweakregstep.asdf. 

In [99]:
result3_myparfile = pipe3.call('jw42424001001_01101_00001_nrca5_cal.fits',
                               config_file='my_custom_parfile_tweakregstep.asdf')

2021-04-05 00:33:10,895 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-04-05 00:33:11,066 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_pars-tweakregstep_0006.asdf
2021-04-05 00:33:11,076 - stpipe - INFO - PARS-TWEAKREGSTEP parameters are {'brightest': 100, 'catalog_format': 'ecsv', 'enforce_user_order': False, 'expand_refcat': False, 'fitgeometry': 'general', 'kernel_fwhm': 2.302, 'minobj': 15, 'nclip': 3, 'save_catalogs': False, 'searchrad': 1.0, 'separation': 0.5, 'sigma': 3.0, 'snr_threshold': 10, 'tolerance': 1.0, 'use2dhist': True, 'xoffset': 0.0, 'yof

ValidationError: Extra value 'brightest' in root

The last example showed how to use the `call` method in conjunction with a custom parameter file to configure the pipeline. This is the reccomended approach.

It is possible to accomplish the same thing using the `run` method - in this case, we will have to set every value in the parameter file that we wish to change from the coded default to an attribute directly on the pipeline object. 

In [115]:
pipe3 = Image3Pipeline()


pipe3.brightest = 50
pipe3.kernel_fwhm = 2.302
pipe3.minobj = 15
pipe3.nclip = 2
pipe3.searchrad = 1.0
pipe3.separation = 0.5
pipe3.sigma = 3.0
pipe3.snr_threshold = 5


pipe3.run('jw42424001001_01101_00001_nrca5_cal.fits')

2021-04-05 09:54:24,921 - stpipe - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-04-05 09:54:24,976 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2021-04-05 09:54:24,979 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2021-04-05 09:54:24,983 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2021-04-05 09:54:24,990 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2021-04-05 09:54:24,994 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2021-04-05 09:54:24,996 - stpipe.Image3Pipeline.resample - INFO - R

'\n    '

## Overriding Reference Files in Python

As shown above in example 1 (link), the `model.meta` (which is derived from the `fits` file header) contains paths to the reference files that will be used by the pipeline. Changing these file names in the file header will point the pipeline to use a different reference file. Often times users will want to do this in order to substitute in their own reference files. Note that the header value must contain the full path to the reference file, and due to character limits a full path is often too long (hence the reason the CRDS_PATH environment variable is set to shorten the path in the header). 

#### Example x: Substituting Reference Files in Python

In this example, we will run `Detector1Pipeline` on the `uncal.fits`, but substitute the CRDS-chosen dark reference file for our own, `my_dark.fits`.

In [131]:
pipe1_mydarkref = Detector1Pipeline

# put some working code here....

When calling the pipeline object directly using `pipe.run`, the desired reference file overrides can be set directly via the `override_<reffile>` attributes.

In [133]:
pipe1_mydarkref = Detector1Pipeline()
pipe1_mydarkref.override_dark = 'mydark.fits'
result_mydarkref = pipe1_mydarkref('jw42424001001_01101_00001_nrca5_uncal.fits')

2021-04-11 23:43:12,894 - stpipe.Image2Pipeline - WARNING - /Users/cshanahan/miniconda3/envs/jwst-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-04-11 23:43:12,910 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-04-11 23:43:12,912 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-04-11 23:43:12,914 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-04-11 23:43:12,915 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-04-11 23:43:12,917 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-04-11 23:43:12,919 - stpipe.Detector1Pipeline.superbias -

2021-04-11 23:43:13,572 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_linearity_0052.fits'.
2021-04-11 23:43:13,574 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_mask_0038.fits'.
2021-04-11 23:43:13,574 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_persat_0016.fits'.
2021-04-11 23:43:13,575 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE reference file is '/Users/cshanahan/crds_cache/references/jwst/nircam/jwst_nircam_readnoise_0042.fits'.
2021-04-11 23:43:13,577 - stpipe.Detector1Pipeline - INFO - Prefetch for REFPIX reference file is 'N/A'.
2021-04-11 23:43:13,578 - stpipe.Detector1Pipeline - INFO - Prefetch for RESET reference file is 'N/A'.
2021-04-11 23:43:13,578 - stpipe.Detector1Pipeline - INFO -

2021-04-11 23:43:16,723 - stpipe.Detector1Pipeline.refpix - INFO - side_smoothing_length = 11
2021-04-11 23:43:16,723 - stpipe.Detector1Pipeline.refpix - INFO - side_gain = 1.000000
2021-04-11 23:43:16,724 - stpipe.Detector1Pipeline.refpix - INFO - odd_even_rows = True
2021-04-11 23:43:18,646 - stpipe.Detector1Pipeline.refpix - INFO - Step refpix done
2021-04-11 23:43:18,783 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(1, 7, 2048, 2048) from jw42424001001_01101_00001_nrca5_uncal.fits>,).
2021-04-11 23:43:18,785 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2021-04-11 23:43:18,809 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /Users/

2021-04-11 23:44:02,172 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'gain_scale', 'search_output_file': True, 'input_dir': ''}
2021-04-11 23:44:02,223 - stpipe.Detector1Pipeline.gain_scale - INFO - GAINFACT not found in gain reference file
2021-04-11 23:44:02,224 - stpipe.Detector1Pipeline.gain_scale - INFO - Step will be skipped
2021-04-11 23:44:02,226 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale done
2021-04-11 23:44:02,368 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<CubeModel(1, 2048, 2048) from jw42424001001_01101_00001_nrca5_uncal.fits>,).
2021-04-11 23:44:02,370 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': No

In [135]:
result_mydarkref.meta.ref_file.instance

{'crds': {'sw_version': '10.3.5', 'context_used': 'jwst_0704.pmap'},
 'dark': {'name': 'crds://jwst_nircam_dark_0043.fits'},
 'gain': {'name': 'crds://jwst_nircam_gain_0056.fits'},
 'ipc': {'name': 'crds://jwst_nircam_ipc_0028.fits'},
 'linearity': {'name': 'crds://jwst_nircam_linearity_0052.fits'},
 'mask': {'name': 'crds://jwst_nircam_mask_0038.fits'},
 'persat': {'name': 'crds://jwst_nircam_persat_0016.fits'},
 'readnoise': {'name': 'crds://jwst_nircam_readnoise_0042.fits'},
 'saturation': {'name': 'crds://jwst_nircam_saturation_0063.fits'},
 'superbias': {'name': 'crds://jwst_nircam_superbias_0022.fits'},
 'trapdensity': {'name': 'crds://jwst_nircam_trapdensity_0004.fits'},
 'trappars': {'name': 'crds://jwst_nircam_trappars_0002.fits'}}

## Skipping Pipeline Steps in Python

In Example 1, we saw that the `meta` section of a DataModel shows which pipeline steps were run/skipped to produce that file. This can be set when running the pipeline in Python. The following will create a `Detector1Pipeline` with the `DarkCurrentStep` switched off. Because this is being set directly on the pipeline, this setting will only be used if we call this pipeline directly (vs. using `.call`).

In [137]:
pipe_skipdark = Detector1Pipeline()
pipe_skipdark.dark_current.skip = True


2021-04-12 09:16:31,375 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-04-12 09:16:31,378 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-04-12 09:16:31,381 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-04-12 09:16:31,383 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-04-12 09:16:31,385 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-04-12 09:16:31,387 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-04-12 09:16:31,391 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-04-12 09:16:31,394 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-04-12 09:16:31,396 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2021-04-12 09:16:31,430 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2021-04-12 0

# Running the Pipeline Using the Command Line Interface (stpipe run) <a class="anchor" id="7"></a>

The previous section demonstrated how to run the JWST pipeline (and steps, individually), as well as how to configure parameters, reference files, and which steps to run/skip, in Python. All of these same tasks are possible via the command line `stpipe run` command. The most basic way to use command line interface (CLI) is:

    $ ./stpipe run -h
    usage: stpipe run [-h] [--config <path>] [-d] [-p <name>=<value>] [-v] <class> <input-file>

    positional arguments:
      <class>            step or pipeline class name (case-insensitive, module path may be omitted for unique class names)
      <input-file>       input dataset or association

    optional arguments:
      -h, --help         show this help message and exit
      --config <path>    config file (use 'stpipe print-config' to save and edit the default config)
      -d, --debug        debug logging (DEBUG level)
      -p <name>=<value>  override an individual step or pipeline parameter (use 'stpipe describe' to list available parameters)
      -v, --verbose      verbose logging (INFO level)
    
The first argument can either be the Python class name of the desired pipeline/step, or a path to a parameter file. The second argument is the input file name.

#### Example 1: Running Detector1Pipeline on a NIRCAM Image <a class="anchor" id="7a"></a>

The following is the command line equivilent to [Example 1](#5a). This will run the `Detector1Pipeline` on the uncal.fits file with the CRDS designated parameter file, and all best reference files chosen by CRDS.

In your shell:

    >> $ stpipe run jwst.pipeline.Detector1Pipeline jw42424001001_01101_00001_nrca5_uncal.fits
    
#### Example 10: Running a Single Pipeline Step (LinearityStep) on a NIRCAM Image (#7b) <a class="anchor" id="7b"></a>

The CLI can also be used to run a single pipeline step, just like in python. The equivilant to [Example 2](#5b) would be:

    >> $ stpipe run jwst.linearity.LinearityStep jw42424001001_01101_00001_nrca5_uncal.fits
    
#### CLI tools

There are several tools available in `stpipe` to help navigate the available pipelines/steps and options for running them.


    1. stpipe list - explore available pipeline/step class names
    
          To list available pipeline classes:

                >> $ stpipe list

          To list pipelines and steps:

               >> $ stpipe list --all

          To list steps in the jwst package:

               >> $stpipe list --steps-only jwst.*
    
     2. stpipe describe - describes pipeline/steps in detail.
     
          To print step description and parameters:

               >> $ stpipe describe jwst.step.RampFitStep

          To increase verbosity to include parameters for nested steps:

               >> stpipe describe --verbose jwst.pipeline.Detector1Pipeline
               
     3. stpipe print-config - print step or pipeline config to stdout
     
          To save a pipeline config to a local file:

               >> stpipe print-config jwst.pipeline.Detector1Pipeline dataset.fits > config.asdf
    